In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging
from IPython.display import Markdown, display

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-01-19 10:55:44.097433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-19 10:55:44.097484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-19 10:55:44.098336: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 10:55:44.104745: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-19 10:55:44.850765: W tensorflow/compiler/tf2

In [2]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-01-19 10:55:47,946 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-01-19 10:55:47,948 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [3]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-01-19 10:55:50,022 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-01-19 10:55:50,022 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


# Configuration

In [ ]:
# Workflow instance
cur_workflow = Workflow(
    name="Create slected image datasets",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/multimodal/selected_image_datasets.ipynb",
    workflow_type="multimodal_feature_engineering"
)
# Configuration instance.

# Set to False if you only need the metadata from the bag, and not the assets.
download_assets = True

config = ExecutionConfiguration(
    # Comment out the following line if you don't need the assets.
    # datasets=[source_dataset] if download_assets else [],
    assets=['4-3S3G'],
    workflow=cur_workflow,
    description="Create slected image datasets on dev")

# Initialize execution
execution = EA.create_execution(config)



In [ ]:
print(execution)

In [ ]:
feature = pd.read_csv(execution.asset_paths[0])
feature.dropna(subset=['Image'], inplace=True)
feature.drop_duplicates(subset=['Image'], inplace=True)

selected_image = feature['Image'].tolist()

In [ ]:
eye_ai = EA.domain_path
ml = EA.pathBuilder.schema['deriva-ml']
dataset = ml.Dataset

dataset_sub = eye_ai.Subject_Dataset
sub = eye_ai.Subject
obs = eye_ai.Observation
image = eye_ai.Image

result = dataset.path.filter(dataset.RID == '2-7P5P').\
link(dataset_sub, on=dataset.RID==dataset_sub.Dataset).\
link(sub, on=dataset_sub.Subject==sub.RID).\
link(obs, on=sub.RID==obs.Subject).\
link(image, on=obs.RID==image.Observation)
catalog_image = pd.DataFrame(result.entities())[['RID', 'MD5']]

In [ ]:
# Ingest new values to the feature table
EA.add_features(angle_feature_list)
EA.add_features(lat_feature_list)



# Upload results

In [ ]:
# save assets to asset_path

# upload assets to catalog
execution.upload_execution_outputs(clean_folder=True)